In [1]:
import torch
import time
from diffusers import StableDiffusionPipeline

/home/seulgi/anaconda3/envs/dreambooth/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipeline = StableDiffusionPipeline.from_pretrained(
    "/input/model/path",
    torch_dtype=torch.float16).to("cuda")

Loading pipeline components...: 100%|██████████████████████████████████████| 7/7 [00:26<00:00,  3.77s/it]


In [3]:
def measure_sd_latency(pipe, prompt, num_images=1, repeat=10):
    # Warm-up
    _ = pipe(prompt, num_images_per_prompt=num_images)
    torch.cuda.synchronize() # Wait until GPU operations are complete

    # Clear memory tracking
    torch.cuda.reset_peak_memory_stats() # Reset peak memory usage from previous runs
    
    start = time.time()
    for _ in range(repeat):
        _ = pipe(prompt, num_images_per_prompt=num_images)
    torch.cuda.synchronize()
    end = time.time()
    
    avg_latency = (end - start) / repeat
    peak_memory = torch.cuda.max_memory_allocated() / 1024**2
    
    return avg_latency, peak_memory

In [8]:
latency, memory = measure_sd_latency(pipeline, "a sks backpack on a wooden table" , num_images=1, repeat=30)
print(f"Avg latency: {latency*1000:.2f} ms")
print(f"Peak GPU memory: {memory:.2f} MB")

100%|████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 25.63it/s]

Avg latency: 2028.93 ms
Peak GPU memory: 3269.06 MB


In [5]:
def measure_sd_latency_event(pipe, prompt, num_images=1, repeat=10):
    pipe(prompt, num_images_per_prompt=num_images) # wram-up
    torch.cuda.synchronize()
    torch.cuda.reset_peak_memory_stats()

    total_time = 0.0

    for _ in range(repeat):
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)

        start_event.record()
        _ = pipe(prompt, num_images_per_prompt=num_images)
        end_event.record()

        torch.cuda.synchronize()
        elapsed = start_event.elapsed_time(end_event)
        total_time += elapsed

    avg_latency = total_time / repeat
    peak_memory = torch.cuda.max_memory_allocated() / 1024**2

    return avg_latency, peak_memory

In [9]:
latency, memory = measure_sd_latency_event(pipeline, "a sks backpack on a wooden table" , num_images=1, repeat=30)
print(f"Avg latency: {latency:.2f} ms") # No need to multiply by 1000 since the result is already in milliseconds
print(f"Peak GPU memory: {memory:.2f} MB")

100%|████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 25.75it/s]

Avg latency: 2023.91 ms
Peak GPU memory: 3269.06 MB


In [7]:
batch_sizes = [1, 2, 4, 8]
prompt = "a sks backpack on a wooden table"

for batch_size in batch_sizes:

    print(f"\nBatch size: {batch_size}")

    # Warm-up run
    _ = pipeline(prompt, num_images_per_prompt=batch_size)

    # Start measurement
    torch.cuda.synchronize()
    start = time.time()
    _ = pipeline(prompt, num_images_per_prompt=batch_size)
    torch.cuda.synchronize()
    end = time.time()

    elapsed = (end - start)
    elapsed_ms = elapsed * 1000
    fps = batch_size / elapsed
    per_image_time = elapsed_ms / batch_size

    print(f"Time: {elapsed_ms:.2f}ms | FPS: {fps:.2f} | Per image: {per_image_time:.2f}ms")


Batch size: 1


100%|████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 25.75it/s]


Time: 2018.63ms | FPS: 0.50 | Per image: 2018.63ms

Batch size: 2


100%|████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 16.39it/s]


Time: 3192.50ms | FPS: 0.63 | Per image: 1596.25ms

Batch size: 4


100%|████████████████████████████████████████████████████████████████████| 50/50 [00:05<00:00,  8.39it/s]


Time: 6226.56ms | FPS: 0.64 | Per image: 1556.64ms

Batch size: 8


100%|████████████████████████████████████████████████████████████████████| 50/50 [00:11<00:00,  4.50it/s]


Time: 11633.50ms | FPS: 0.69 | Per image: 1454.19ms
